In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, Sequential, losses, optimizers, metrics
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

encoder_handle = r'model/bert_encoder/'
preprocesser_handle = r'model/bert_preprocessor/'

In [2]:
df = pd.read_csv(r'../dataset/online_shopping_10_cats.csv')
class_names = list(df.cat.drop_duplicates())
class2idx = {}
idx2class = {}
for idx, name in enumerate(class_names):
    class2idx[name] = idx
    idx2class[idx] = name

In [3]:
# 自定义预处理模型
def bert_preprocessor(sentence_features, seq_length=128):
    text_inputs = [layers.Input(shape=(), dtype=tf.string, name=ft)
                   for ft in sentence_features]  # 处理输入的句子特征
    
    preprocessor = hub.load(preprocesser_handle)
    tokenize = hub.KerasLayer(preprocessor.tokenize, name='tokenizer')
    tokenized_inputs = [tokenize(segment) for segment in text_inputs]  # 将句子划分为字
    
    packer = hub.KerasLayer(
        preprocessor.bert_pack_inputs,
        arguments=dict(seq_length=seq_length),
        name='packer'
    )
    encoder_inputs = packer(tokenized_inputs)
    return keras.Model(text_inputs, encoder_inputs, name='preprocessor')
preprocessor = bert_preprocessor(['input1'])

In [4]:
def build_classifier():
    text_input = layers.Input(shape=(), dtype=tf.string, name='input')
    text_preprocessed = preprocessor(text_input)
    encoder = hub.KerasLayer(encoder_handle, trainable=True, name='BERT_encoder')
    x = encoder(text_preprocessed)['pooled_output']
    x = layers.Dropout(0.3)(x)
    x1 = layers.Dense(1, name='emotion')(x)
    x2 = layers.Dense(10, name='classifier')(x)
    return keras.Model(text_input, [x1, x2])
classifier_model = build_classifier()

In [140]:
# classifier_model.load_weights(r'./checkpoints2/bert_classifier_seq_256')
classifier_model.load_weights(r'./checkpoints2/bert_classifier_epoch2')

In [125]:
def binary_classifier(out):
    ret = []
    for item in out:
        ret.append(1 if item[0] > 0.5 else 0)
    return ret

def multi_classifier(out, num=3):  # 输出前num个的类别
    ret = []
    for item in out:
        classes = []
        arg = np.argsort(item.numpy())[::-1]
        for i in range(num):
            classes.append((idx2class[arg[i]], np.round(item[arg[i]].numpy(), 2)))
        ret.append(classes)
    return ret

x = ["写的真好，期待后续出版", "这几本确实都很不错", "烧水速度真快",
     "有点悬念外，其他内容都很空洞", "缺少对人物的心理活动描写", "这书送的也太慢了", "运行速度太快了，非常好",
     "华为系统挺好", "适合看视屏用", "小米的也很好用", "垃圾，自带的软件都删不掉", "根本连不上wifi",
     "玩两天王者就卡了，根本操作不起来", "机身太大了，打电话没声音", "电池根本用不了几天", "好",
     "原来卖4000现在卖1000块，太坑人了吧？", "很好打折后只要100块，血赚", "真的甜，好吃!", "烟台的苹果真好",
     "发过来全烂了", "个头太小了吧，塞牙缝都不够", "这东西都放了多久，都是烂的，这也敢卖", "一盒能用一个月，真好用",
     "用了后顺滑多了，头发也没油了", "和沐浴露加起来10块，真好", "两瓶合起来50，太便宜了", "用来洗头真不错",
     "cpu太慢了吧", "键盘发过来就是个坏的", "这自带的杀毒软件把自己整垮了", "性价比很高，三星的内存条速度很快",
     "位置不错离景点蛮近的，早餐也很好吃", "房子虽然不大，但是很精细，服务也很好", "除了没有浴缸和保险柜，其他都很好",
     "什么破地方，连个wifi都没有", "餐饮真的差，又贵又难吃", "靠近马路，隔音效果太差，根本睡不着", "尺码太小了，穿的很不舒服",
     "蒙牛真的垃圾!", "蒙牛的新包装不错", "在平板上买的洗发水很好用", "平板上订的酒店，比手机上订的贵", "衣服上印的手机很好看",
     "酒店里的平板很慢", "酒店里的平板比我手机慢", "这衣服搭配平板比搭配手机漂亮"]
pad = ['卖家对包装挺负责人的,用起来还还不错',
'屏幕还好啊,应该没有阴阳屏的问题,入手半天,没看出来阴阳屏',
'果然轻多了,  还不错啦',
'东西收到了,很快很不错,就是不知道发票什么时候邮过来',
'轻薄时尚,靓丽炫酷。',
'很好  苹果的东西一直都支持的  就是价格贵呀',
'玩具感很强烈 没有ipad2那么有质感',
'东西很不错,虽然有一点阴阳,但用着用着就完全忘了,不影响使用',
'帮别人买的,用着据说不错',
'很好,给朋友买的很喜欢']
phone = ['用了几天,就是感觉喇叭的音质不好!听歌,接打电话,声音效果都很一般,价我收到的手机是12年4月份出厂的,怪不得价格这么便宜,原因就是卖不出去……',
'发货速度很快,态度比较好,但是手机内存没有4G,只有不到2G,对这点比较失望。',
'商家发货速度较快,手机为行货,性价比不错',
'g网加补丁才可以上网,但是网络奇差无比 电信肯定故意的,我现在就想去办电信卡',
'下单,两天后到货。机子不错,是正品行货。',
'主要硬件足够使用,屏幕大小适合。屏幕清晰度总体还可以,在阳光下表现一般。很少发生死机。主要不足:通话声音太小,不知道中兴为什么把手机最主要的功能把忽略了,如果手机通话不行,还有谁买这玩意,有点奇怪的是用免提时声音表现并不小。 音质很一般,还好我不听歌。',
'这个价格,也没什么好说的,一般般吧',
'手机比较满意 希望电池能用长一点',
'优点:手机款式我挺喜欢的,携带比较方便。价位比较合适,觉得挺划算的! 缺点:手机反应优点慢,电池每天都得充电,比较麻烦!',
'老妈在用,待机时间也还可以。']
wash = ['很好,很给力!!!!!!!!!!!!',
'在这里还是要帮亚马逊说句话,买之前看到很多评论不太好,但是我还是坚持买了。 收到货看到外包装有些不平整,心里开始有些担心,但是里面的包装是完好无损的,没有拆过。而且我觉得净重量也差不多的,味道也对的,所以是正品!',
'比超市便宜很多,很划算,味道挺好闻的,用了一下还好',
'洗发水太稀了,而且邮回来的时候瓶子上全是灰。用了一个月,每次洗完头发很涩,以前我完全没有头皮屑的,现在有很多头皮屑。已经停用换其它牌子了。就晚了一天不能退货了。',
'清凉舒适,不错,夏天用超合适',
'这个还没开始用  原来用的也是这个  要是感觉再说!',
'我跟同事一人买了一瓶。大包装~用了以后头发很柔顺头皮也很舒服,整体感觉不错',
'瓶子很好,东西是真的。',
'第二次买了,很好用哦',
'使用后感觉不错,值得拥有。']
heater = ['帮岳父买的,买了俩,他家两套房子出租,房客要求装的。到目前没说有啥问题。',
'已经用了一个月了,感觉不错。没有那么多华而不实的功能,很简洁,很实惠。夏天可以开机即洗,天凉后的加热和保温效果还有待尝试。性价比很高的一款热水器。',
'海尔ES80H-Z3电热水器是新品,增加了3D功能,价格比实体店便宜很多,就是安装材料费用有点贵200元。刚开始用不知道耗不耗电。',
'质量很好服务不错。安好一直在用',
'比商场卖的便宜很多啊  质量还是不错的  售后服务一样好',
'给了个5星,确实不错,我是从电热水器即热式改换过来的,海尔3d速热跟即热式可谓天壤之别,海尔3d冬天带给你那叫一个爽。没得说,一年内把家里的两个全换了,前几天买的第三个是买给小叔子的。',
'升温挺快的,方便快捷',
'很好用够大,安装服务回访都不错',
'热水器蛮好用的,发货也很快,就安装一波三折最后终于安装好了']
bool_computer = ['帮忙别人买的书 书本从各种介绍来说都是被称为一本经典的数据库概念书 但是本来国内翻译国外的书籍就是一件极其有风险的事情 书籍很经典但是翻译的却很糟糕是个很普遍的现象- 而且还有国内外的阅读习惯的差异问题 但是这本书更大问题是 极度糟糕的排版 字体偏小 英文大量使用小斜体 看起来就像一堆混乱的蚂蚁 书本除了一行一行读下去以为没有其他的阅读方式- 太糟糕了 作为一名数据库工程师 对书中全面全方位的讲解表示不错 从SQL直接开始然后才是数据库设计这一点表示很新颖(这里全方位指的是翻译5版没有压缩的那一版-不是本科教学版  不过东西都一样) 对于现在的浮躁的要死的大学生里面 有的狂妄的SB用高中学习数学的方法 天天做题 自以为自己很NB 不让他做题跟你急的这种人 不解释 数据库本来就是一门概念理解起来很难的课程 我也是学习了很多年以后才开始学习数据库的 从初学者的角度来看的话 这本书讲的有点杂乱-  另外书中关于规范化的部分的讲解方式 我个人不是很适应 我们老师就曾说过 讲完这门课以后 学生回家听别人谈论数据库的时候能够知道讲的名词是什么东西 -不至于一头雾水就不错可以了',
'买这本书是因为我看了很多魔乐的视频教程,觉得不错,书的印刷质量不错,内容也挺合适,关键是自己喜欢去学。。。。',
'书也还不错, 只是太忙了没时间去看啊..怨念',
'是正品的,目前还没看,书的印刷等非常好',
'这本书很简短,很普通,还这么贵,感觉很不值,很多内容讲的和模糊',
'书有缺页现象  收货的时候也没有办法发现 ..哎 懒的换了',
'很好!很有用处,很快就到了~',
'发货速度还行,书的质量也可以,就是油墨似乎未干的感觉,有的地方摸得用力了点会留下脏色。 总体来说这本书还是很满意的。',
'恩,书还是很不错的,就是价格有点高,不过买300减100,你懂的。。质量还可一的,不错,以后慢慢的看吧。。,',
'还不错看起来  就是比想象中大一些 厚一些']
x = computer
emotion, classes = classifier_model(tf.constant(x), training=False)
emotion = binary_classifier(emotion)  # 转化为正负情感
classes = multi_classifier(classes, num=3)  # 显示排名前三的类别
for i in range(len(x)):
    print(f"\"{x[i]}\"：{emotion[i]}，{classes[i]}")

"帮忙别人买的书 书本从各种介绍来说都是被称为一本经典的数据库概念书 但是本来国内翻译国外的书籍就是一件极其有风险的事情 书籍很经典但是翻译的却很糟糕是个很普遍的现象- 而且还有国内外的阅读习惯的差异问题 但是这本书更大问题是 极度糟糕的排版 字体偏小 英文大量使用小斜体 看起来就像一堆混乱的蚂蚁 书本除了一行一行读下去以为没有其他的阅读方式- 太糟糕了 作为一名数据库工程师 对书中全面全方位的讲解表示不错 从SQL直接开始然后才是数据库设计这一点表示很新颖(这里全方位指的是翻译5版没有压缩的那一版-不是本科教学版  不过东西都一样) 对于现在的浮躁的要死的大学生里面 有的狂妄的SB用高中学习数学的方法 天天做题 自以为自己很NB 不让他做题跟你急的这种人 不解释 数据库本来就是一门概念理解起来很难的课程 我也是学习了很多年以后才开始学习数据库的 从初学者的角度来看的话 这本书讲的有点杂乱-  另外书中关于规范化的部分的讲解方式 我个人不是很适应 我们老师就曾说过 讲完这门课以后 学生回家听别人谈论数据库的时候能够知道讲的名词是什么东西 -不至于一头雾水就不错可以了"：0，[('书籍', 10.38), ('水果', 0.55), ('平板', 0.53)]
"买这本书是因为我看了很多魔乐的视频教程,觉得不错,书的印刷质量不错,内容也挺合适,关键是自己喜欢去学。。。。"：1，[('书籍', 11.67), ('水果', 0.48), ('洗发水', -0.25)]
"书也还不错, 只是太忙了没时间去看啊..怨念"：0，[('书籍', 11.13), ('酒店', 0.33), ('衣服', -0.27)]
"是正品的,目前还没看,书的印刷等非常好"：0，[('书籍', 10.35), ('洗发水', 0.57), ('衣服', -0.04)]
"这本书很简短,很普通,还这么贵,感觉很不值,很多内容讲的和模糊"：0，[('书籍', 9.57), ('计算机', -0.13), ('衣服', -0.3)]
"书有缺页现象  收货的时候也没有办法发现 ..哎 懒的换了"：0，[('书籍', 9.66), ('衣服', 0.82), ('计算机', -0.2)]
"很好!很有用处,很快就到了~"：1，[('平板', 3.53), ('手机', 2.74), ('计算机', 

### 使用Amazon数据集验证

1. `rating.csv` 中存储了评论和打分，通过productId可找到对应的商品.
2. `products.csv` 中存储了productId对应的商品的名称和分类的编号catIds.
3. `categories.csv` 中存储了每种catId对应的类别名称.

使用方法：首先在`categories.cvs`中找到目标类别对应的catId，然后在`products.csv`中找到包含该分类编号的商品，最后从`rating.csv`中找到对应的评论.

In [12]:
amazon_df = pd.read_csv('../dataset/yf_amazon/ratings.csv')
products_df = pd.read_csv('../dataset/yf_amazon/products.csv')
cats_df = pd.read_csv('../dataset/yf_amazon/categories.csv')

In [13]:
print('总数据数', amazon_df.shape[0])
print('空评论的数目', sum(amazon_df['comment'].isna()))
amazon_df = amazon_df[amazon_df['comment'].notna()]
print('处理后剩余数目', amazon_df.shape[0])

总数据数 7202921
空评论的数目 3121753
处理后剩余数目 4081168


In [129]:
cat_names = ['书籍', '平板', '手机', '水果', '洗发水', '热水器', '蒙牛', '衣服', '计算机', '酒店']
cats_df[cats_df.apply(lambda row: '电脑' in row['category'], axis=1)]  # 找每个类别名称对应的编号

,catId,category
89,89,电脑包
187,187,电脑元件
205,205,电脑整机
250,250,MIDI乐器/电脑音乐
501,501,其它电脑周边
642,642,平板电脑
645,645,USB电脑周边
761,761,平板电脑配件
818,818,电脑硬件
1057,1057,电脑/办公


In [130]:
# 衣服：内衣，羊绒衫，卫衣，风衣，毛衣，大衣，速干衣，棉衣，皮衣，连衣裙，冲锋衣，运动球衣
cats = [[832], [642], [304], [-1], [1121], [702], [-1],
        [1100, 965, 903, 899, 875, 717, 585, 569, 534, 331, 245, 90], [1057], [-1]]  # 需要类别名称对应的类别编号
rate = {}
for cat, name in zip(cats, cat_names):
    def check(row):
        items = [int(a) for a in row['catIds'].split(',')]  # 将类别用逗号划分开
        for item in items:
            if item in cat:
                return True
        return False
    products = products_df[products_df.apply(check, axis=1)]['productId']
    item_df = amazon_df[amazon_df.apply(lambda row: row['productId'] in products, axis=1)][['rating', 'comment']]
    print(f"'{name}'类别中商品数目{products.shape[0]}，评论数目{item_df.shape[0]}")
    item_df = item_df.sort_values(by=['rating'], ascending=False, ignore_index=True)  # 按评分排序保存到文本中
    item_df.to_csv(f"./validation/{name}.csv", index=False)

'计算机'类别中商品数目20112，评论数目359804


In [137]:
# 构造验证集，3分及以下为负面评论，4分及以上为正面评论
info_df = pd.DataFrame(columns=['类别', '总数目', '正例', '负例'])
val_x, val_y = [], []
x = []
for name in cat_names:
    df = pd.read_csv(f"./validation/{name}.csv")
    if df.shape[0] > 15000:  # 每种类别上限10000项
        df = df.sample(15000)
    pos_num = df[df['rating']>3].shape[0]  # 正例数目
    neg_num = df[df['rating']<=2].shape[0]  # 负例数目
    tot_num = pos_num + neg_num
    print(f"类别'{name}', 总数目{tot_num}, 正例{pos_num}, 负例{neg_num}")
    info_df.loc[info_df.shape[0]] = [name, tot_num, pos_num, neg_num]
    def add(row):
        if row['rating'] == 3:  # 如果是3分认为是中等评论，不纳入数据集
            return
        emotion = 1 if row['rating'] > 3 else 0
        x.append(row['comment'])
        val_x.append(tf.constant(row['comment'], tf.string))
        val_y.append((emotion, class2idx[name]))
    df.apply(add, axis=1)
val_ds = tf.data.Dataset.from_tensor_slices((val_x, val_y))
info_df

处理类别'书籍', 总数目13282, 正例12156, 负例1126
处理类别'平板', 总数目12058, 正例10870, 负例1188
处理类别'手机', 总数目13218, 正例11429, 负例1789
处理类别'水果', 总数目0, 正例0, 负例0
处理类别'洗发水', 总数目12357, 正例11098, 负例1259
处理类别'热水器', 总数目3539, 正例3264, 负例275
处理类别'蒙牛', 总数目0, 正例0, 负例0
处理类别'衣服', 总数目0, 正例0, 负例0
处理类别'计算机', 总数目13367, 正例12154, 负例1213
处理类别'酒店', 总数目0, 正例0, 负例0


,类别,总数目,正例,负例
0,书籍,13282,12156,1126
1,平板,12058,10870,1188
2,手机,13218,11429,1789
3,水果,0,0,0
4,洗发水,12357,11098,1259
5,热水器,3539,3264,275
6,蒙牛,0,0,0
7,衣服,0,0,0
8,计算机,13367,12154,1213
9,酒店,0,0,0


In [ ]:
emotion_acc = keras.metrics.BinaryAccuracy('emotion_acc')  # 情感分类上的准确率
class_acc = keras.metrics.SparseCategoricalAccuracy('class_acc')  # 物品分类上的准确率
for (x, y) in tqdm(val_ds.batch(32)):
    emotion_y = tf.reshape(y[:, 0], [-1, 1])  # 情感标签
    classes_y = tf.reshape(y[:, 1], [-1, 1])  # 分类标签
    emotion, classes = classifier_model(x, training=False)
    emotion_acc.update_state(emotion_y, emotion)
    class_acc.update_state(classes_y, classes)
    break
print(f"情感分类准确率: {emotion_acc.result().numpy():.2%}")
print(f"商品分类准确率: {class_acc.result().numpy():.2%}")
print(f"总计验证数目：{emotion_acc.count.numpy()}")